In [1]:
data_var = '2024-01-14'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11979,2024-01-14,Brasil Nbb,11:00,Botafogo,Paulistano,2.76,1.41,157.5,1.85,1.85,4.5,2.00,1.72,4ztghInK,0.362319,0.709220,0.540541,0.540541,0.071539,0.8,0.6,NaN,NaN,302.164,77.290460,0.255790,115.502,33.372309,0.288933,275.606,113.526,278.40,95.20,0.0,0.0,0.0,0.0,0.457839,0.000000,0.106446,2.35,0.470,3.744681,0.0,0.0,0.0,2.43,0.486,0.843621,0.000000,0.0,0.000000
11980,2024-01-14,Brasil Nbb,18:00,Franca,Unifacisa,1.18,4.47,163.5,1.84,1.86,-11.5,1.97,1.75,naVcix2Q,0.847458,0.223714,0.543478,0.537634,0.071171,0.4,0.8,NaN,NaN,132.748,23.152366,0.174408,191.204,93.228162,0.487585,97.402,198.018,82.40,319.01,0.0,0.0,0.0,0.0,0.823498,0.007644,0.083636,-0.81,-0.162,-1.111111,0.0,0.0,0.0,-2.18,-0.436,-7.958716,0.538463,0.4,-0.138463
11981,2024-01-14,Espanha Acb,08:30,Granada,Obradoiro CAB,1.61,2.33,165.5,1.85,1.95,-4.5,2.03,1.75,MNG7oBGN,0.621118,0.429185,0.540541,0.512821,0.050303,0.0,0.6,NaN,NaN,367.830,172.644990,0.469361,336.222,232.616558,0.691854,0.000,344.276,149.33,729.00,0.0,1.0,0.0,0.0,0.258435,0.037216,0.104757,0.00,0.000,inf,0.0,0.0,0.0,1.82,0.364,3.653846,0.000000,0.0,0.000000
11982,2024-01-14,Espanha Acb,08:30,Murcia,Real Madrid,3.29,1.34,166.5,1.85,1.91,5.5,2.04,1.72,4n6CpVVT,0.303951,0.746269,0.540541,0.523560,0.050220,0.4,0.6,NaN,NaN,130.364,10.176418,0.078062,124.940,33.905989,0.271378,142.842,122.592,126.75,179.19,0.0,0.0,0.0,0.0,0.595619,0.022567,0.120359,0.28,0.056,40.892857,0.0,0.0,0.0,1.66,0.332,1.024096,0.000000,0.0,0.000000
11983,2024-01-14,Espanha Acb,13:00,Manresa,Baskonia,2.15,1.71,170.5,1.89,1.90,1.5,2.01,1.78,0AucNwTs,0.465116,0.584795,0.529101,0.526316,0.049912,0.6,0.0,NaN,NaN,300.932,209.969929,0.697732,148.614,42.450392,0.285642,136.158,0.000,173.40,125.55,0.0,0.0,0.0,0.0,0.161206,0.003731,0.085823,0.91,0.182,6.318681,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12156,2024-01-14,Polônia Liga Feminina De Basquete,14:00,Polonia W. F,Sosnowiec F,3.37,1.29,149.5,1.81,1.89,5.5,2.03,1.67,GtG1cP6N,0.296736,0.775194,0.552486,0.529101,0.071930,0.8,0.6,NaN,NaN,457.636,277.619662,0.606639,117.788,38.922356,0.330444,243.108,200.970,679.00,86.94,1.0,0.0,0.0,0.0,0.631237,0.030578,0.137599,-2.19,-0.438,-5.410959,0.0,0.0,0.0,-1.09,-0.218,-1.330275,0.000000,0.0,0.000000
12157,2024-01-14,Reino Unido Troféu Da Bbl,15:00,Cheshire Phoenix,Plymouth City Patriots,1.15,4.98,172.5,1.85,1.85,-14.5,2.08,1.63,v9lEGNnf,0.869565,0.200803,0.540541,0.540541,0.070368,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.883595,0.000000,0.171535,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
12158,2024-01-14,Reino Unido Torneio Feminino Betty Codona,15:00,Manchester Giants F,Nottingham Wildcats F,1.06,7.70,155.5,1.83,1.87,-15.5,1.88,1.82,6aYUM2mg,0.943396,0.129870,0.546448,0.534759,0.073266,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,908.50,0.0,1.0,0.0,0.0,1.071961,0.015289,0.022933,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
12159,2024-01-14,República Tcheca Nbl,14:00,Usti n. Labem,Srsni Pisek,1.01,12.20,178.5,1.87,1.79,-17.5,2.03,1.70,U72t8nC6,0.990099,0.081967,0.534759,0.558659,0.072066,0.8,0.4,NaN,NaN,116.944,12.316044,0.105316,388.298,256.780854,0.661298,176.858,481.858,131.

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,17:30,Eua Nba,Denver Nuggets,Indiana Pacers,237.5,1.83,0.9999,Over
1,21:00,Eua Nba,Milwaukee Bucks,Sacramento Kings,246.5,1.83,0.9998,Over
2,21:00,Eua Nba,Minnesota Timberwolves,Los Angeles Clippers,222.5,1.86,0.9979,Over
3,23:00,Eua Nba,Portland Trail Blazers,Phoenix Suns,230.5,1.89,1.0000,Over
4,19:00,Eua Ncaa,Stanford,Utah Utes,155.5,1.91,0.8397,Over
